In [ ]:
path_to_muograph = "../"
import sys
sys.path.insert(1, path_to_muograph)

![alternative text](../images/muograph_logo.png)


**<h1><center>Tracking: tutorial</center></h1>**

The `TrackingMST` class is used to process **muon tracks** in the context of m**uon scattering tomography**. 

It combines the tracking **feature** of **muons** detected **before** and **after** the **volume of interest** and compute features relevant to **muon scattering tomography** reconstruction algorithms such as **scattering angle**.


The initialization is done by providing 2 instances of the `Tracking` class: 

- The **incoming muons** (muons detected **before** interaction with the volume of interest).

-  The **outgoing muons** (muons detected **after** interaction with the volume of interest).

First, let's gather the **incoming muon hits**. In the input data file, **panels** are **labeled** from **top to bottom**: 

 - **Labels 0, 1, 2** correspond to the **detector panels** placed **before** the **volume of interest**.

 - **Labels 3, 4, 5** correspond to the **detector panels** placed **after**.

In [ ]:
csv_file = '../muograph/data/iron_barrel/barrel_and_cubes_scattering.csv'

from muograph.hits.hits import Hits

hits_in = Hits(
    csv_filename = csv_file,  # The csv file
    plane_labels = (0, 1, 2),  # Include hits from first planes 0, 1 and 2
    input_unit = "mm",  # The distance unit in the csv file.
    spatial_res = (1.0, 1.0, 0.),  # The spatial resolution along x, y and z in mm.
    efficiency = 0.90,
)
hits_in

In [ ]:
hits_out = Hits(
    csv_filename = csv_file,  # The csv file
    plane_labels = (3, 4, 5),  # Include hits from first planes 0, 1 and 2
    input_unit = "mm",  # The distance unit in the csv file.
    spatial_res = (1.0, 1.0, 0.),  # The spatial resolution along x, y and z in mm.
    efficiency = 0.90,
)
hits_out

We can now initialize the respective `Tracking`:

In [ ]:
from muograph.tracking.tracking import Tracking

tracks_in = Tracking(label="above", hits=hits_in)
tracks_out = Tracking(label="below", hits=hits_out)

tracks_in, tracks_out

The `TrackingMST` class **takes** the following **arguments**:

- `trackings (Optional[Tuple[Tracking, Tracking]])`: instances of the Tracking class
for the incoming muon tracks (Tracking.label = 'above') and outgoing tracks
(Tracking.label = 'below')


- `output_dir (Optional[str])`: Path to a directory where to save TrackingMST attributes
in a hdf5 file.

In [ ]:
output_dir = "../output_test/"

from muograph.tracking.tracking import TrackingMST
mst = TrackingMST(trackings=(tracks_in, tracks_out))
mst

### **Efficiency**

**Efficiencies** of the **incoming** and **outgoing** **tracks** (`tracks_in.tracks_eff` and `tracks_out.tracks_eff`) are **combined** to compute the event-wise efficiency `mst.muon_eff`.

In [ ]:
tracks_in.tracks_eff, tracks_out.tracks_eff

If both `tracks_in.tracks_eff` and `tracks_out.tracks_eff` = 1, `mst.muon_eff` is set to `True`. Otherwise, it is set to `False`.

Only muons with `muon_eff == True` are kept. 

In [ ]:
(tracks_in.tracks_eff.bool() & tracks_out.tracks_eff.bool()).sum(), mst.muon_eff.sum()

AS expected, the resulting **tracking efficiency** is the **product** of the **incoming** and **outgoing** tracks **efficiency**.

In [ ]:
tracks_in.tracking_eff * tracks_out.tracking_eff, mst.tracking_eff

The `TrackingMST` class **combines** the **features** of both `Tracking` instances:

### **Tracks**

In [ ]:
event = 8114
f"Muon #{event} incoming direction px, py, pz = {mst.tracks_in[event, 0]:.3f}, {mst.tracks_in[event, 1]:.3f}, {mst.tracks_in[event, 2]:.3f}"

In [ ]:
f"Muon #{event} outgoing direction px, py, pz = {mst.tracks_out[event, 0]:.3f}, {mst.tracks_in[event, 1]:.3f}, {mst.tracks_in[event, 2]:.3f}"

### **Points**

In [ ]:
f"Muon #{event} incoming position x, y, z = {mst.points_in[event, 0]:.3f}, {mst.points_in[event, 1]:.3f}, {mst.points_in[event, 2]:.3f} [mm]"

In [ ]:
f"Muon #{event} incoming position x, y, z = {mst.points_out[event, 0]:.3f}, {mst.points_out[event, 1]:.3f}, {mst.points_out[event, 2]:.3f} [mm]"

### **Angles**

**Angles** are expressed in **radiants**.

#### **Zenith angles**

In [ ]:
f"Muon #{event} has incoming zenith angle = {mst.theta_in[event]:.3f} [rad]"

In [ ]:
f"Muon #{event} has outgoing zenith angle = {mst.theta_out[event]:.3f} [rad]"


In [ ]:
from muograph.plotting.plotting import plot_hist
import torch
plot_hist(mst.theta_in if mst.theta_in.device == torch.device("cpu") else mst.theta_in.cpu() , xlabel = r"$\theta_{in}$ [rad]")

In [ ]:
from muograph.plotting.plotting import plot_hist
plot_hist(mst.theta_out if mst.theta_out.device == torch.device("cpu") else mst.theta_out.cpu() , xlabel = r"$\theta_{out}$ [rad]" )

#### **Scattering angle**

The muon scatteing angle $\Delta\theta$ is computed from the inmcopming and outgoing tracks direction $\overrightarrow{T}_{in} = (p_{x, in}, p_{y, in}, p_{z, in})$ and $\overrightarrow{T}_{out} = (p_{x, out}, p_{y, out}, p_{z, out})$:

$$
\Delta\theta = cos^{-1}\left(\frac{\overrightarrow{T}_{in} \cdot \overrightarrow{T}_{out}}{|\overrightarrow{T}_{in}| |\overrightarrow{T}_{out}|}\right)
$$

In [ ]:
f"Muon #{event} has scattering angle = {mst.dtheta[event]:.4f} [rad]"


In [ ]:
plot_hist(mst.dtheta if mst.dtheta.device == torch.device("cpu") else mst.dtheta.cpu() , xlabel = r"$\Delta\theta$ [rad]",  logy=True)

#### **Summary**

The **zenith angle**, **energy** and **scattering angle distributions** can be **plotted** using the `plot_muon_features` function. 

If a file name is provided as `figname`, the **figure** is **saved** as the desired location.

In [ ]:
mst.plot_muon_features(figname=output_dir+"theta_dtheta_E")

#### **Plot event**

The **fitted tracks** can be **plotted** using the `TrackingMST.plot_tracking_event()` function. 


Similarly to `Tracking.plot_tracking_event()` it displays the fitted points and tracks on a 2D projection.

In [ ]:
mst.plot_tracking_event(
    proj="YZ", # The desired projection
    event=event, # The event to plot
    figname="../output_test/tracking_event_mst_YZ", # Name of the figure to save
    )

mst.plot_tracking_event(
    proj="XZ", # The desired projection
    event=event,  # The event to plot
    figname="../output_test/tracking_event_mst_XZ", # Name of the figure to save
    )

If an instance of the `Volume` class interest is provided, the **volume of interest** gets represented on the plot: 

In [ ]:
from muograph.volume.volume import Volume
import numpy as np
voi = Volume(
    position = (0, 0, -1200),
    dimension = (1000, 600, 600),
    voxel_width = 20)

mst.plot_tracking_event(
    voi=voi, # The Volume of interest
    proj="YZ",
    event=np.random.randint(10000),
    figname="../output_test/tracking_event_mst_voi")